
<h1><center>Trends in Forest Recovery After Stand Replacing Disturbance: A Spatiotemporal Evaluation of Productivity in Southeastern Pine Forests</center></h1>

<center> Google Earth Engine Python API Processing Script <center>

<h4><center> Daniel J. Putnam </center></h4>

## Analysis Preperation

### _Libraries_

In [ ]:
import geemap
import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
from datetime import datetime
%matplotlib inline
import scipy as sp
import scipy.signal as scisig
ee.Initialize()

In [ ]:
## used for google earth engine API authentication if token expires
#ee.Authenticate(auth_mode='paste')
#ee.Authenticate()

### _Imports_

In [ ]:
LS5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2") # landsat 5
LS7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2") # landsat 7
LS8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") # landsat 8
LCMS = ee.ImageCollection("USFS/GTAC/LCMS/v2021-7") # landscape Change Monitoring System
NLCD_col = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD") # National Landcover Database
loblolly = ee.FeatureCollection("users/dputnam21/us_eco_l3_NEW") # manually selected ecoregions for study area

### _Priliminary set-up_

In [ ]:
# Creating sample date range for disturbances
startingD = ee.Date.fromYMD(1989,1,1)
endingD = ee.Date.fromYMD(2011,12,31)

### _Landsat Preprocessing_

In [ ]:
# Cloud masking based on the QA band : code adapted from Landsat example in data catalog in GEE
def LScloudMask(image):
  qa = image.select('QA_PIXEL')
    # removing cloud pixels if confiance is high, cloud shadow, snow
  cloud = qa.bitwiseAnd(1 << 3).And(qa.bitwiseAnd(1 << 9)) \
            .Or(qa.bitwiseAnd(1 << 4)) \
            .Or(qa.bitwiseAnd(1 << 5))
  return image.updateMask(cloud.Not())

## A function that applies scaling factors and offsets : code adapted from Landsat example in data catalog in GEE
def applyScaleFactors(image):
  opticalBands = image.select(ee.List.sequence(0,10)).multiply(0.0000275).add(-0.2)
  return image.addBands(opticalBands, None, True)

# adding the cloud mask per generation
LS5 = LS5.map(LScloudMask)
LS7 = LS7.map(LScloudMask)
LS8 = LS8.map(LScloudMask)

# applying scaling factors
LS5 = LS5.map(applyScaleFactors)
LS7 = LS7.map(applyScaleFactors)
LS8 = LS8.map(applyScaleFactors)

# Landsat 5/7 & 8 differ in their band labeling, bands need to be renamed them to
# match each other before merging collections    
LS8BandNames = ee.List(['SR_B4','SR_B3','SR_B5','SR_B6','SR_B7','QA_PIXEL'])
NewBandNames = ee.List(['SR_B3','SR_B2','SR_B4','SR_B5','SR_B7','QA_PIXEL'])
LS8 = LS8.select(LS8BandNames,NewBandNames)

# merging the Landsat 5 and 7 collections
LS_stack = LS5.merge(LS8)
LS_stack = LS_stack.merge(LS7)

# data reduction on the image stack
LS_stack = LS_stack.filterBounds(loblolly)

In [ ]:
# Adding a function to calculate and add an NBR band for a single image.
def addNBR(image):
  nbr = image.normalizedDifference(['SR_B4', 'SR_B7']).rename('NBR')
  return image.addBands(nbr)

# Adding tNBR to the filtered combined Landsat collection
LS_stack_wVI = LS_stack.map(addNBR)

---

## Stand Identification Method

### _Landcover/Landuse Mask_

In [ ]:
# New NLCD/LCMS method
# retrieve NLCD for each year
NLCD_2001 = NLCD_col.filter(ee.Filter.eq('system:index', '2001')).first().select("landcover")
NLCD_2004 = NLCD_col.filter(ee.Filter.eq('system:index', '2004')).first().select("landcover")
NLCD_2006 = NLCD_col.filter(ee.Filter.eq('system:index', '2006')).first().select("landcover")
NLCD_2008 = NLCD_col.filter(ee.Filter.eq('system:index', '2008')).first().select("landcover")
NLCD_2011 = NLCD_col.filter(ee.Filter.eq('system:index', '2011')).first().select("landcover")
NLCD_2013 = NLCD_col.filter(ee.Filter.eq('system:index', '2013')).first().select("landcover")
NLCD_2016 = NLCD_col.filter(ee.Filter.eq('system:index', '2016')).first().select("landcover")
NLCD_2019 = NLCD_col.filter(ee.Filter.eq('system:index', '2019')).first().select("landcover")

# combine NLCD to image collection
NLCDlandcover_col = ee.ImageCollection( \
                    ee.List([NLCD_2001,NLCD_2004,NLCD_2006,NLCD_2008, \
                             NLCD_2011,NLCD_2013,NLCD_2016,NLCD_2019]))

# Function to remap NLCD classes of interest for conditional layer
def remapNLCD(image):
    image = ee.Image(image)
    image = image.updateMask(ee.Image.constant(42).Or(ee.Image.constant(52)))
    image = image.remap(ee.List([42,52]),ee.List([10,1]),defaultValue = None)
    return image

# Layer containing the summed values of pixels across the collection after remapping
NLCDclassSum = NLCDlandcover_col.map(remapNLCD).reduce(ee.Reducer.sum())
NLCDMask = NLCDclassSum.remap(ee.List([62,71,80]),ee.List([1,1,1]), defaultValue = None)

In [ ]:
# retrieve LCMS landuse classification
LCMSlanduseCol = LCMS.select("Land_Use")

# A function to select only forest landuse class
def remapLCMS(image):
    image = ee.Image(image)
    onlyForest = image.remap([3],[1], defaultValue = None)
    return onlyForest

LCMSlanduseSum = LCMSlanduseCol.map(remapLCMS).reduce(ee.Reducer.sum())

# # combining the two layers into a landuse / landcover mask
lulcMask = NLCDMask.updateMask(LCMSlanduseSum.gte(36))
lulcMask = lulcMask.clip(loblolly) # clip mask to study boundaries for better loading

### _LCMS Fast Change Method_

In [ ]:
# Using the LCMS Change metric to identify harvest areas
LCMSchange = LCMS.select('Change_Raw_Probability_Fast_Loss')

def LCMSchangeSelection(image):
    # filtering pixels to only > 70% fast change confidence
    image = ee.Image(image)
    minConfidence = 70
    gtePercent = image.gte(ee.Image.constant(minConfidence))
    gtePercent = gtePercent.updateMask(gtePercent.eq(1))
    # updating the image properties for disturbance year
    gtePercent = gtePercent.set({'year':image.date().get('year')})
    outImage = gtePercent.updateMask(lulcMask).rename('remapped')
    return outImage

# applying the function to the LCMS
FC_stack = LCMSchange.map(LCMSchangeSelection)

### _Connected Pixel (Min stand size) mask_

In [ ]:
# A function to apply a connected pixel mask to the input image
def conectPixls(InImage,minArea,maxPixels):
    pixelCount = InImage.connectedPixelCount(maxPixels,False)
    minPixelCount = ee.Image(minArea).divide(ee.Image.pixelArea())
    outImage = InImage.updateMask(pixelCount.gte(minPixelCount))
    return outImage

# a function to be mapped accross an image collection and annually apply the connected pixels mask, also creates an
# additional band to store the year of disturbance for each pixel
def annualConectPixls(image):
    conectPixlsMasked = conectPixls(image,40000,1024) # minimum stand size of 4 ha (represented in m3), and
    imgYear = image.get('year')                       #     maximum of 92 ha (represented in pixel count) (tool limit)
    imgYearBand = ee.Image.constant(imgYear).uint16().rename('ChangeY')
    imgYearBand = imgYearBand.updateMask(conectPixlsMasked)
    return conectPixlsMasked.addBands(imgYearBand)

FC_final = FC_stack.map(annualConectPixls)

In [ ]:
# creating the summary images
FC_final_changeN = FC_final.select('remapped').reduce(ee.Reducer.sum())
FC_final_firstYear = FC_final.select('ChangeY').reduce(ee.Reducer.min())
FC_final_lastYear = FC_final.select('ChangeY').reduce(ee.Reducer.max())

### _Applying disturbance window mask_

In [ ]:
# Going to help to keep detected disturbances within a given window of time
# first detected disturbance
FC_final_changeN = FC_final_changeN.updateMask(FC_final_firstYear.gte(startingD.get('year')) \
                                               .And(FC_final_firstYear.lte(endingD.get('year')))
                                              )
# last detected disturbance
FC_final_changeN = FC_final_changeN.updateMask(FC_final_lastYear.gte(startingD.get('year')) \
                                               .And(FC_final_lastYear.lte(endingD.get('year')))
                                              )


In [ ]:
# Final potential sample pixels
# An image representing pixels that meet all selection criteria
potentialSamples = ee.Image.toUint8(FC_final_changeN.updateMask(FC_final_changeN.eq(1))).rename('remapped_sum')

### _Filter Selection to Only Include Homogenous, Non-Edge Groups of Pixels_

In [ ]:
# Edge avoidence
PS_connectedPixelCount = potentialSamples.reduceNeighborhood(ee.Reducer.count(),
                                                             ee.Kernel.circle(2, 'pixels', False, 1),
                                                             'mask',
                                                             True
                                                            )
potentialSamples2 = potentialSamples.updateMask(PS_connectedPixelCount.gte(13))

---

## Automatic Stand Selection Method

### _Creating Sampling Areas Using Ecoregions_

In [ ]:
# Function to convert the ecoregion code to an integer value
def convertPropertyToBand(feat):
    feat = ee.Feature(feat)
    prop = feat.get('US_L3CODE')
    propInt = ee.Number.parse(prop).toInt()
    feat = feat.set({'numericL3ecocode':propInt})
    return feat
loblolly = loblolly.map(convertPropertyToBand)

# Need to convert ecoregion feature collection and the property to integer in order for it to be used 
#     as the 'classBand' in the stratifiedSample fucntion
ecoregionImage = ee.Image(loblolly.reduceToImage(['numericL3ecocode'],ee.Reducer.first()))
ecoregionImage = ecoregionImage.cast({'first':'uint8'})
ecoregionImage = ecoregionImage.clipToCollection(loblolly)

# (old method), want to just export a single band
# Adding ecoregion code as band to potential sample pixels
#potentialSamples2 = potentialSamples2.addBands(ecoregionImage.select('first').rename('numericL3ecocode'))

potentialSamples2 = ecoregionImage.select('first').rename('numericL3ecocode').updateMask(potentialSamples2)

### _Imports/Exports of Created Data_

In [ ]:
## today's date
today = str(datetime.now()).split(" ")[0]
today = today.replace("-","_")
today = "_"+today
print(today)

In [ ]:
# # Optional batch export process to export the final potential samples raster
# exportTask = ee.batch.Export.image.toDrive(image = potentialSamples2, 
#                                            description = 'PotentialSamples'+today, 
#                                            folder = 'EarthEngine_Exports', 
#                                            region = loblolly.geometry(), 
#                                            scale = 30, 
#                                            skipEmptyTiles = True,
#                                            maxPixels = 3000000000,
#                                            fileFormat = 'GeoTIFF', 
#                                            )
# exportTask.start()

In [ ]:
## Importing points created in arcpro using the raster exported above, couldn't figure out a way to create sample points
##    in GEE with a minimum 1km spacing in time for thesis
samplePoints = ee.FeatureCollection('users/dputnam21/samplePoints_06282022')

---

### _Creating Random Sample Points_

In [ ]:
## Sample points can be created using this function, however the proportional allocation of samples has to 
##    be done manaually and there is no functionality for minimum point spacing
# samplePoints = potentialSamples2.stratifiedSample(numPoints = 50,
#                                                  classValues = [34,35,45,63,64,65,66,67,68,71,73,74,75],
#                                                  classPoints = [2,404,391,148,1,799,5,40,36,4,1,61,114],
#                                                  region = loblolly,
#                                                  classBand = 'numericL3ecocode',
#                                                  scale = 30,
#                                                  seed = 5,
#                                                  dropNulls = True,
#                                                  geometries = True,
#                                                  )

### _Creating Point Buffers for Sampling_

In [ ]:
# optional reduction in the number of sample points for methodology testing
sampleLimit = samplePoints.size().getInfo() # can change this to a number for data reduction purposes

samplePoints2 = samplePoints.limit(ee.Number(sampleLimit),
                                   'UniqueID',
                                   True
                                  )
numSamples = samplePoints2.size().getInfo()

In [ ]:
# defining a function to be mapped over point feature collection to create buffers
def makeBuffers(feat):
    inFeat = ee.Feature(feat)
    buff = inFeat.buffer(distance = 60) # 60 meter buffer = 2 pixel radius to align with circlular kernel mask
    return buff

sampleCircles = samplePoints2.map(makeBuffers)

### _Displaying streamed layers on a map_

In [ ]:
# LCMS landcover palette
LCMSlcPalette = ['efff6b','ff2ff8','1b9d0c','97ffff','a1a1a1','c2b34a','1B1716']

Map = geemap.Map(basemap="SATELLITE")

# centered on Wadesboro NC, the location of a figure in the thesis
Map.centerObject(ee.Feature(ee.Geometry.Point([-80.076728, 34.966592])),13)

# This is the bottom of the layer order

# Each layer is an additive mask, meaning the name of the layer is just the step in a cumulative process

Map.addLayer(ee.Image().paint(loblolly, color = 'black',width = 3), name = 'ecoRegion Outlines', shown = False)
Map.addLayer(ecoregionImage.select('first'), vis_params = {'palette': LCMSlcPalette, 'min': 45, 'max':75}, name = 'Ecoregion Code Image',shown = False)
Map.addLayer(NLCDMask, vis_params = {'palette': ['ffffcc'],'min':1,'max':1}, name = 'NLCD landcover Mask', shown = True)
Map.addLayer(lulcMask, vis_params = {'palette': ['a1dab4'],'min':1,'max':1}, name = 'LCMS landuse mask', shown = True)
Map.addLayer(FC_stack, vis_params = {'palette': ['41b6c4'],'min':0,'max':1}, name = 'Fast Change (>70%)', shown = True)
Map.addLayer(FC_final.select('remapped'), vis_params = {'palette': ['2c7fb8'],'min':0,'max':1}, name = 'Minimum Disturbance Area', shown = True)
Map.addLayer(potentialSamples2.select('numericL3ecocode'),{'palette':['7E3054'],'min':0,'max':1}, name = 'Edge Removal, Only 1 Fast Change', shown = True)
Map.addLayer(samplePoints,{'color':'d9c31c'}, name = 'Stratified Random Samples',shown = False)

# This is the top of the layer order

Map.addLayerControl()
Map

---

## NBR Time-Series Extraction

### _Setup for Annual Compositing & Time-series Extraction Procedure_

In [ ]:
# enter analysis parameters
compositeMonthStart = 2 
compositeMonthEnd = 3 #inclusive
outputIndex = 'NBR'

# prep for function
chart_VI = LS_stack_wVI.filter(ee.Filter.calendarRange(compositeMonthStart,compositeMonthEnd,'month'));
proj = potentialSamples2.select('numericL3ecocode').projection()

years = ee.List.sequence(1984, 2021)

In [ ]:
# Interting a single constant image into the Landsat stack to avoid missing data in 1990
#    the 0.0 value is replaced with an NA object later, GEE doesn't like mapping over NA's
zeroImage1 = ee.Image.constant(0.0).clipToBoundsAndScale(loblolly.geometry(),scale = 30).toFloat()
zeroImage1 = zeroImage1.rename(['NBR'])
zeroImage = zeroImage1.set('system:time_start', ee.Date.fromYMD(1990,compositeMonthStart,1).millis())

chart_VI = chart_VI.merge(zeroImage)

### _Vegetation Index Extraction Method_

In [ ]:
# Extraction function that creates annual composites, writes values to a feature collection
def annualComposite (year):
    filteredColl = chart_VI.filter(ee.Filter.calendarRange(year, year, 'year'))
    singleImage = filteredColl.select([str(outputIndex)]).reduce(ee.Reducer.median()) ## CHANGE COMPOSITE STAT HERE ###
    outputImage = singleImage.set('system:time_start', ee.Date.fromYMD(year,compositeMonthStart, 1).millis())
    outputCollection = outputImage.reduceRegions(collection = sampleCircles,
                                                 reducer = ee.Reducer.mean(),
                                                 scale = 30
                                                )
    return outputCollection

export_data = ee.FeatureCollection(years.map(annualComposite)).flatten()

In [ ]:
# Running a batch export process to execute the processes above
exportTask = ee.batch.Export.table.toDrive(collection = export_data,
                                            description = (outputIndex+"_values"+today),
                                            folder = 'EarthEngine_Exports',
                                            fileFormat = 'CSV',
                                            selectors = ['UniqueID','mean']
                                            )
exportTask.start()

---

## Interpolating and Reformatting Data Locally

### _Reformatting, and Writing Extracted VI Data_

In [ ]:
# Importing the data exported in the above cell
# It's easier to download locally then load here than to upload to GEE for access
importVI = pd.read_csv('C:/R_workspace/Collection2_data/'+outputIndex+'_values_2022_09_22.csv')

In [ ]:
# reformatting above values into dataframe (one row for each stand, 38 columns for each year)

standIDs = np.sort(importVI.iloc[:,0].unique())
imageYears = years.getInfo()

# Creating the dataframe
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
VItsDF = pd.DataFrame(index = standIDs, columns = imageYears)

for stand in standIDs:
    singleStandDF = importVI.loc[importVI['UniqueID'] == stand]
    VItsDF.iloc[stand,:] = singleStandDF['mean']
    
VItsDF[1990] = [np.nan]*len(standIDs)

# for some reason all the columns dtype is 'object' converting to float above wasn't working 
#      so I'll just have to convert the columns
for col in VItsDF:
    VItsDF[col] = pd.to_numeric(VItsDF[col], errors='coerce')
VItsDF

### _Inter-annual Interpolation and creating time series plots_

In [ ]:
## interpolation of null values using Akima splines
plotDF = VItsDF.interpolate(axis = 'columns',method = 'akima')

# setting up for time-series plots
fig, axs = plt.subplots(7, 3, sharex=False, sharey=True, figsize = (16,25))

for i, ax in enumerate(fig.axes):
    ax.plot(plotDF.iloc[i,].transpose(),label = "Interpolated")
    ax.plot(VItsDF.iloc[i,].transpose(),label = "Original")
    ax.scatter(imageYears,VItsDF.iloc[i,], color = 'orange',s = 20)
    ax.legend(loc='lower right')
    ax.set_title("Stand"+' '+str(standIDs[i])+' '+"Time-Series")
    
fig.subplots_adjust(left=0.1, bottom=0.1, right=None, top=None, wspace=0.3, hspace=0.5)
fig.text(0.5, 0.04, 'Year', ha='center', va='center')
fig.text(0.06, 0.5, (outputIndex+' '+'value'), ha='center', va='center', rotation='vertical')

fig.savefig("C:/R_workspace/timeSeries_interpolation.svg")

In [ ]:
# display the interpolated table
plotDF

# write the table locally
plotDF.to_csv(path_or_buf="C:/R_workspace/Collection2_data/"+outputIndex+"_timeSeries"+today+".csv", \
              sep=',', na_rep='', float_format=None, header=True, index=True, mode='w')